In [1]:
from tfl.client import Client
from tfl.api_token import ApiToken

app_id = 'Subscription_Peterson'
app_key = '8d1f8ab05c7c49db81dd99c40f70574c'

token = ApiToken(app_id, app_key)

client = Client(token)
#print (client.get_line_meta_modes())
#print (client.get_lines(mode="bus")[0])
#print (client.get_lines(line_id="victoria")[0])

In [2]:
import requests
from collections import defaultdict

In [3]:
import urllib.request, json
import requests

try:
    url = "https://api.tfl.gov.uk/crowding/940GZZLUSKS/Live?app_key=8d1f8ab05c7c49db81dd99c40f70574c"
    hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    }

    req = urllib.request.Request(url, headers=hdr)
    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    print(response.getcode())
#    print(response.read())
except Exception as e:
    print(e)

200


In [4]:
import pandas as pd

In [5]:
# use requests to interact with an API
import requests, zipfile, io

# The API path should be stated in the documentation 
#api_path = 'https://api.tfl.gov.uk/journey/journeyresults/westminster/to/bank'
api_path = "https://api.tfl.gov.uk/crowding/940GZZLUSKS/Live?app_key=8d1f8ab05c7c49db81dd99c40f70574c"

tube_lines = [("bakerloo","#B36305"),("central","#E32017"),("circle","#FFD300"),("district","#00782A"),("hammersmith-city","#F3A9BB"),("jubilee","#A0A5A9"),("metropolitan","#9B0056"),("northern","#000000"),("piccadilly","#003688"),("victoria","#0098D4"),("waterloo-city","#95CDBA")]
#tube_lines = {"bakerloo":"#B36305","central":"#E32017","circle":"#FFD300","district":"#00782A","hammersmith-city":"#F3A9BB","jubilee":"#A0A5A9","metropolitan":"#9B0056","northern":"#000000","piccadilly":"#003688","victoria":"#0098D4","waterloo-city":"#95CDBA"}

stations = defaultdict(list)

#api_path = "https://api.tfl.gov.uk/crowding/940GZZLUSKS/Wed?app_key=8d1f8ab05c7c49db81dd99c40f70574c"

for line, color in tube_lines:
    api_path1 = "https://api.tfl.gov.uk/Line/%s/StopPoints" % line
    r1 = requests.get(api_path1).json()
    
    for entry in r1:
        stations[entry["naptanId"]] = [line, color, entry["commonName"][:-20]]      

url_coord = "https://api.tfl.gov.uk/stationdata/tfl-stationdata-detailed.zip"

r = requests.get(url_coord)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("./details")

df = pd.read_csv("./details/StationPoints.csv")

req = urllib.request.Request(url_coord, headers=hdr)

req.get_method = lambda: 'GET'
response = urllib.request.urlopen(req)

defaultdict(<class 'list'>, {'940GZZLUBST': ['metropolitan', '#9B0056', 'Baker Street'], '940GZZLUCHX': ['northern', '#000000', 'Charing Cross'], '940GZZLUEAC': ['northern', '#000000', 'Elephant & Castle'], '940GZZLUEMB': ['northern', '#000000', 'Embankment'], '940GZZLUERB': ['bakerloo', '#B36305', 'Edgware Road (Bakerloo)'], '940GZZLUHAW': ['bakerloo', '#B36305', 'Harrow & Wealdstone'], '940GZZLUHSN': ['bakerloo', '#B36305', 'Harlesden'], '940GZZLUKEN': ['bakerloo', '#B36305', 'Kenton'], '940GZZLUKPK': ['bakerloo', '#B36305', 'Kilburn Park'], '940GZZLUKSL': ['bakerloo', '#B36305', 'Kensal Green'], '940GZZLULBN': ['bakerloo', '#B36305', 'Lambeth North'], '940GZZLUMVL': ['bakerloo', '#B36305', 'Maida Vale'], '940GZZLUMYB': ['bakerloo', '#B36305', 'Marylebone'], '940GZZLUNWY': ['bakerloo', '#B36305', 'North Wembley'], '940GZZLUOXC': ['victoria', '#0098D4', 'Oxford Circus'], '940GZZLUPAC': ['district', '#00782A', 'Paddington'], '940GZZLUPCC': ['piccadilly', '#003688', 'Piccadilly Circus']

In [6]:
i = 0
for stop in df.StationUniqueId:
    i +=1
    if stop in list(stations.keys()) and df.FriendlyName.iloc[i].startswith(('Sub','Met','MET')):
        
        stations[stop].append(df.Lat.iloc[i])
        stations[stop].append(df.Lon.iloc[i])

In [7]:
day_station = {'Mon':{},'Tue':{},'Wed':{},'Thu':{},"Fri":{},'Sat':{},'Sun':{}}

for day in day_station.keys():

        
    for station in stations.keys():    
        # api_path = "https://api.tfl.gov.uk/crowding/940GZZLUSKS/Live?app_key=8d1f8ab05c7c49db81dd99c40f70574c"
        api_path = "https://api.tfl.gov.uk/crowding/%s/%s?app_key=8d1f8ab05c7c49db81dd99c40f70574c" % (station, day)
    
        r = requests.get(api_path).json()
        try:
            #print(r["timeBands"][1]["percentageOfBaseLine"])
            day_station[day][station] = r["timeBands"][48]["percentageOfBaseLine"]
        except:
            continue
        
        #print(r['percentageOfBaseline'])

In [8]:
from geopy.geocoders import ArcGIS
import folium

import ipywidgets as widgets
from IPython.display import display, clear_output

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="location_details")

location = geolocator.geocode("London")
print((location.latitude, location.longitude))

def create_map(dayofweek):
    # Clear previous output to avoid multiple maps showing up
    clear_output(wait=True)
    
    # Create a map centered on the initial position
    mymap = folium.Map(location=[location.latitude, location.longitude], zoom_start=13)

    # Add markers based on the current time_step
    for naptan, station in stations.items():
        try:
            coef = day_station[dayofweek][naptan]
            popup_info = station[2]+'\n'+str(coef)
            try:
                folium.Circle(location=[station[3],station[4]],radius=500*coef,popup=popup_info,color=station[1],fill=True,fill_color=station[1],fill_opacity=0.4).add_to(mymap)
            except:
                continue
        except:
            continue
                
    display(mymap)

(51.5074456, -0.1277653)


In [9]:
day_slider = widgets.SelectionSlider(
    options=list(day_station.keys()),
    value='Mon',  # Default value
    description='Day:',
    continuous_update=False  # Update map only when slider is released
)

# Display the map initially
create_map('Mon')

def on_day_change(change):
    create_map(change['new'])

day_slider.observe(on_day_change, names='value')

# Display the slider
display(day_slider)

SelectionSlider(continuous_update=False, description='Day:', options=('Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'…

In [10]:
mymap1=folium.Map(location=[location.latitude, location.longitude],zoom_start=15)

import folium.plugins as plugins

for naptan, station in stations.items():
    
    api_path = "https://api.tfl.gov.uk/crowding/%s/Live?app_key=8d1f8ab05c7c49db81dd99c40f70574c" % naptan
    
    r = requests.get(api_path).json()
    coef = r['percentageOfBaseline']
    popup_info = station[2]+'\n'+str(coef)
    
    try:        
        folium.Circle(location=[station[3],station[4]],radius=500*coef,popup=popup_info,color=station[1],fill=True,fill_color=station[1],fill_opacity=0.4).add_to(mymap1)
    except:
        continue

mymap1

In [11]:
#mymap.add_child(folium.Marker(location=[location.latitude, location.longitude],popup=location,icon=folium.Icon(color='blue')))